In [1]:
# Main program for snp

from ib_insync import *

import logging
import pandas as pd
from os import listdir

from helper import get_connected, grp_opts, get_prec, catch
from snp_func import p_snps, p_snpopts

from snp_variables import *

util.startLoop()  # should not be there in py file

keep_pickles = True   # keep already pickled symbols

ib = get_connected('snp', 'live') # close the connection and delete it final function

Peer closed connection
clientId 1 already in use?
API connection failed: TimeoutError()


In [2]:
# Weekend update functions
# __________________________

def weekend_process(ib):
    '''Weekend process to generate underlyings (lot+margin) and optio pickles
    Arg: (ib) as connection
    Returns: 
        None
        But pickles to _lot_margin.pickle & (SYMBOL).pkl'''
    
    util.logToFile(fspath+'_errors.log')  # create log file
    with open(fspath+'_errors.log', 'w'): # clear the previous log
        pass
    
    # Step # 1: generate the underlyings
    dfu = p_snps(ib).dropna()

    # Step # 2: generate the option pickles
    [p_snpopts(ib=ib,undContract=u, undPrice=p, lotsize=z, margin=m) 
     for u, p, z, m in zip(dfu.contract, dfu.undPrice, dfu.lot, dfu.margin)]
    
    return None

def pickle_remaining_opts(ib, keep_pickles = True):
    '''Upon any failure, pickle remaining options upon failure
    Arg: 
        (ib) as connection
        <keep_pickles> as True / False for keeping existing pickles
    Returns:
       None'''
    
    util.logToFile(fspath+'_errors.log')  # create log file
    
    # get the underlying dataframe
    dfu = pd.read_pickle(fspath+'_lot_margin.pickle')
    
    if keep_pickles:
        fs = listdir(fspath)
        optsList = [f[:-4] for f in fs if f[-3:] == 'pkl']
        dfr = dfu[~dfu.index.isin(optsList)] # remaining dfs (without existing pickles)
        [p_snpopts(ib=ib,undContract=u, undPrice=p, lotsize=z, margin=m)
         for u, p, z, m in zip(dfr.contract, dfr.undPrice, dfr.lot, dfr.margin)]
    else:
        [p_snpopts(ib=ib,undContract=u, undPrice=p, lotsize=z, margin=m)
         for u, p, z, m in zip(dfu.contract, dfu.undPrice, dfu.lot, dfu.margin)]
        
    return None

In [6]:
%%time
# Run the weekend process
weekend_process(ib)

Wall time: 55.2 s


In [3]:
%%time
pickle_remaining_opts(ib)

Wall time: 1h 1min 23s


In [7]:
dfu = pd.read_pickle(fspath+'_lot_margin.pickle')